[![LibKet](../images/LibKet.png)](https://gitlab.com/mmoelle1/LibKet)
**LibKet - The Quantum Expression Template Library.**
- Repository:    https://gitlab.com/mmoelle1/LibKet/
- Documentation: https://libket.readthedocs.io/
- API docs:      https://mmoelle1.gitlab.io/LibKet/

---

# Tutorial \#2: Quantum computing today and future perspective

## Getting started

Let's include **LibKet**'s main headerfile and include its namespaces. This can take some time, stay tuned.

In [1]:
#include "LibKet.hpp"
using namespace LibKet;
using namespace LibKet::circuits;
using namespace LibKet::filters;
using namespace LibKet::gates;

Let us create a simple quantum expression and evaluate it on different quantum backends

In [2]:
auto expr = cnot(h(sel<0>()), sel<1>(init()));

## Cirq

[Cirq](https://quantumai.google/cirq) is a Python package for writing, manipulating, and optimizing quantum circuits developed by Google. It is seamlessly integrated into **LibKet** using the embedded Python interpreter, so you should not recognize all the technical details going on under the hood.

As for the [QuEST](https://quest.qtechtheory.org) simulator, we simply create a quantum device for two qubits, load the quantum expression into it, and evaluate it.

In [5]:
QDevice<QDeviceType::cirq_simulator, 2> device;
auto result = device(expr).eval(1);

Standard Exception: LIBKET_ERROR Embedded Python interpreter terminated with exception
/home/jovyan/LibKet/libket/QJob.hpp, line 498 (get)

In [ ]:
std::cout << result << std::endl;